In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import pandas as pd

In [3]:
import tensorflow_datasets as tfds

# 1) Importing The Data & Preprocessing

In [5]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
## as_supervised=True will load the dataset in a 2-tuple structure (input, target) 

In [31]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [12]:
num_valid = 0.1*mnist_info.splits['train'].num_examples
num_valid = tf.cast(num_valid, tf.int64)
num_valid 

<tf.Tensor: shape=(), dtype=int64, numpy=6000>

In [14]:
num_test = mnist_info.splits['test'].num_examples
num_test = tf.cast(num_test, tf.int64)
num_test

<tf.Tensor: shape=(), dtype=int64, numpy=10000>

### scalling the inputs

In [16]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

In [17]:
scaled_train_and_valid = mnist_train.map(scale)

In [41]:
test_data = mnist_test.map(scale)

In [109]:
# scaling all values from (0,255) to (0,1)

### shuffling the data

In [33]:
buffer_size = 10000
shuffled_train_valid = scaled_train_and_valid.shuffle(buffer_size)

### creating validation and training sets

In [35]:
validation_data = shuffled_train_valid.take(num_valid)

In [36]:
train_data = shuffled_train_valid.skip(num_valid)

### batch size for SGD

In [38]:
batch_size = 100
train_data = train_data.batch(batch_size)

In [39]:
validation_data= validation_data.batch(num_valid)

In [42]:
test_data = test_data.batch(num_test)

In [43]:
validation_inputs, validation_targets = next(iter(validation_data))

# 2) The Model

### stacking the model

In [78]:
input_size = 784
output_size = 10
hidden_layer_1 = 100
hidden_layer_2 = 100
hidden_layer_3 = 100

In [102]:
model = tf.keras.Sequential([
    
                            tf.keras.layers.Flatten(input_shape = (28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_1, activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer_2, activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer_3, activation = 'relu'),
                            tf.keras.layers.Dense(output_size, activation = 'softmax') 
    
                            ])

### optimizer and the loss function

In [103]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'] )

### training the model

In [104]:
max_epoch = 10

In [105]:
model.fit(train_data, epochs = max_epoch, validation_data = (validation_inputs, validation_targets), verbose = 2)

Epoch 1/10
540/540 - 2s - loss: 0.3265 - accuracy: 0.9063 - val_loss: 0.1711 - val_accuracy: 0.9530
Epoch 2/10
540/540 - 2s - loss: 0.1308 - accuracy: 0.9605 - val_loss: 0.1212 - val_accuracy: 0.9650
Epoch 3/10
540/540 - 2s - loss: 0.0925 - accuracy: 0.9715 - val_loss: 0.1174 - val_accuracy: 0.9632
Epoch 4/10
540/540 - 2s - loss: 0.0714 - accuracy: 0.9781 - val_loss: 0.0794 - val_accuracy: 0.9752
Epoch 5/10
540/540 - 2s - loss: 0.0582 - accuracy: 0.9811 - val_loss: 0.0656 - val_accuracy: 0.9803
Epoch 6/10
540/540 - 2s - loss: 0.0490 - accuracy: 0.9847 - val_loss: 0.0568 - val_accuracy: 0.9820
Epoch 7/10
540/540 - 2s - loss: 0.0396 - accuracy: 0.9880 - val_loss: 0.0586 - val_accuracy: 0.9813
Epoch 8/10
540/540 - 2s - loss: 0.0363 - accuracy: 0.9882 - val_loss: 0.0494 - val_accuracy: 0.9840
Epoch 9/10
540/540 - 2s - loss: 0.0297 - accuracy: 0.9909 - val_loss: 0.0431 - val_accuracy: 0.9865
Epoch 10/10
540/540 - 2s - loss: 0.0275 - accuracy: 0.9909 - val_loss: 0.0362 - val_accuracy: 0.9882

In [106]:
# accuracy here shows the fraction that model output is equal to target variable.

# 3) Testing the model

In [107]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 8ms/step - loss: 0.0904 - accuracy: 0.9748


In [108]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.09. Test accuracy: 97.48%
